<a id="top"></a>
# Roman Lvl 4 Data Analysis for Time-Domain Astronomy
**************
**************

## Kernel Information

To run this notebook on the Roman Science Platform, please select the "Roman Calibration" kernel at the top right of your window.
**************

## Learning Goals

By the end of this tutorial, you will:

- Create a light curve from Roman simulated data.
- Calculate and extract cosmological parameters from the light curves.
- Create color curves to have a better understanding of the color of the object.
**************

## Introduction

This notebook is designed to provide a simple analysis of Level 4 imaging data products for the Roman Space Telescope with a focus on time-domain astronomy. Herre, we are leveraging the Open Universe (OU) simulated images, however, you can use any other data as well. The notebook processes and analyzes OU data to provide users with advanced data products that can support transient science, including supernovae, variable stars, and other time-sensitive phenomena.
**************

# Table of Contents

1. [Loading data](#1-input)
   - Imports
   - Reading the input data
   - Preparing the input data
2. [Light curves](#2-LCs)
   - Plotting a single light curve
   - Plotting a light curve of all filters
   - Finding the peak brightness and time for each filter
   - Computing the rise time and decline rate for each filter
   - Fitting the light curves using sncosmo
3. [Color curves](#3-CCs)
   - Creating color curves
   - Plotting several color curves
4. [Additional resources](#4-ADs)
5. [About this notebook](#5-About)
6. [Citations](#6-Cite)

**************
# 1. Loading data <a class="anchor" id="1-input"></a>

## Imports

Here we import the required packages for our data access examples including:
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting
- *sncosmo*  for modeling and fitting light curves
- *numpy* to handle array functions

In [ ]:
# Numerical operations
import numpy as np  # For efficient numerical computations and array handling

# Visualization
import matplotlib.pyplot as plt  # For creating plots and figures

# Astronomy-specific libraries
from astropy.table import Table  # For table manipulation and stacking
import sncosmo  # For modeling and fitting supernova light curves

## Reading the Input Data

This section reads an ECSV file containing the photometric dataset for a random transient from the Open Univserse (OU) Roman simulatins. Details on how the following file was created is shown in another notebook called "[Enter name and link of the notebook here]" which is also available on the Roman Science Platform (RSP). 

The follwoing file contains columns such as mjd-obs, flux, filter, and other metadata.

For more info on how to access the available data on the RSP see [this notebook].

The fluxes were obtained using aperture photometry on the transient. For more info on aperture photmetry see [this other notebook].

In [ ]:
# Load the input data
# This allows users to provide their own data in Astropy Table format
LC_table = Table.read('Updated_Matching_Observations.ecsv', format='ascii.ecsv')

# Filter out rows with missing flux values
# Missing flux values could indicate incomplete or unreliable data points.
LC_table = LC_table[~LC_table['flux'].mask]

In [ ]:
# If you can't access the above dataset or you just want to test and run the rest of the notebook, you can use an example photmetric dataset from a package called sncosmo as follows (more on sncosmo is detailed later)
# LC_table = sncosmo.load_example_data()

## Preparing the Input Data

sncosmo expects specific column names for light curve data and in astropy table format.

In this step, we rename:

mjd-obs to time (to represent the Modified Julian Date of the observation).

filter to band (the specific bandpass filter used in the observation).

flux remains unchanged as it's already appropriately named.

In [ ]:
# Ensure column names are compatible with sncosmo, therefore compatible with the rest of the notebook
# Rename original columns if needed
required_renames = {
    'mjd-obs': 'time',  # Rename 'mjd-obs' to 'time'
    'filter': 'band',   # Rename 'filter' to 'band'
    'flux': 'flux',     # Ensure 'flux' column exists
}
for old_name, new_name in required_renames.items():
    if old_name in LC_table.colnames:
        LC_table.rename_column(old_name, new_name)

In [ ]:
# Modify filter names to match sncosmo's Roman filter conventions. Uncomment the following only if you are using the Open Universe dataset
# if 'band' in LC_table.colnames:
#     LC_table['band'] = [
#         band.replace(band[0], 'f') if band[0].isalpha() else band for band in LC_table['band']
#     ]

In [ ]:
# Define and assign zero-point values for each filter
zp_values = {
    'f062': 27.5, 'f087': 27.0, 'f106': 26.5, 'f129': 26.0,
    'f146': 25.8, 'f158': 25.6, 'f184': 25.4, 'f213': 25.2
}
if 'zp' not in LC_table.colnames:
    LC_table['zp'] = [zp_values.get(band, np.nan) for band in LC_table['band']]

# Add the magnitude system column
if 'zpsys' not in LC_table.colnames:
    LC_table['zpsys'] = ['ab'] * len(LC_table)

In [ ]:
# Convert flux to AB magnitude only if zpsys is not 'ab'
if 'flux' in LC_table.colnames and 'zp' in LC_table.colnames and 'zpsys' in LC_table.colnames:
    # Identify rows where zpsys is not 'ab'
    non_ab_mask = LC_table['zpsys'] != 'ab'
    
    # Convert only those rows to the AB system
    if np.any(non_ab_mask):  # Check if there are any non-ab rows to avoid unnecessary calculations
        flux_values = np.asarray(LC_table['flux'][non_ab_mask], dtype=float)
        zp_values = np.asarray(LC_table['zp'][non_ab_mask], dtype=float)
        # Convert flux to AB system
        LC_table['flux'][non_ab_mask] = -2.5 * np.log10(flux_values) + zp_values
    
    # Ensure all rows now have zpsys as 'ab'
    LC_table['zpsys'][non_ab_mask] = 'ab'

# Add a 'fluxerr' column if missing (default: 5% of flux)
if 'fluxerr' not in LC_table.colnames:
    if 'flux' in LC_table.colnames:
        LC_table['fluxerr'] = 0.05 * LC_table['flux']
    else:
        raise ValueError("Input data must contain a 'flux' column to calculate 'fluxerr'.")

In [ ]:
# Select only the required columns for sncosmo
required_columns = ['time', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']
for col in required_columns:
    if col not in LC_table.colnames:
        raise ValueError(f"Required column '{col}' is missing in the input data.")

lc_data = LC_table[required_columns]

# Save the transformed table to a new ECSV file
output_filename = 'Transformed_for_SNCosmo.ecsv'
lc_data.write(output_filename, format='ascii.ecsv', overwrite=True)

# Display the first few rows of the transformed table
print("Transformed Table for SNCosmo:")
print(lc_data)  # Show the first 10 rows

**************
# 2. Light curves <a class="anchor" id="2-LCs"></a>

## Plotting a single light curve

In [ ]:
# Define the single filter to plot
single_filter_name = 'f062'  # Replace 'f062' with your desired filter name; for example sdssz if you are using the sncosmo sample data

# Select rows corresponding to the single filter
filter_mask = lc_data['band'] == single_filter_name
group = lc_data[filter_mask]

# Check if the filter exists in the data
if not np.any(filter_mask):
    raise ValueError(f"Filter '{single_filter_name}' not found in the data.")
    
# Plot time vs. flux for the selected filter
plt.figure(figsize=(10, 6))
plt.scatter(group['time'], group['flux'], label=single_filter_name)

# Customize the plot
plt.xlabel("MJD (Modified Julian Date)")
plt.ylabel("Flux [AB]")
plt.title(f"Light Curve for Filter: {single_filter_name}")
plt.legend(title="Filter")
plt.gca().invert_yaxis()  # Invert y-axis for magnitudes (lower mags are brighter)
plt.show()

## Plotting a light curve of all filters

In [ ]:
# Group by filters and plot
plt.figure(figsize=(10, 6))

# Get unique filter names
filter_names = np.unique(lc_data['band'])

# Iterate through each filter and plot
for filter_name in filter_names:
    # Select rows corresponding to the current filter
    filter_mask = lc_data['band'] == filter_name
    group = lc_data[filter_mask]
    
    # Plot time vs. flux for the current filter
    plt.scatter(group['time'], group['flux'], label=filter_name)

# Customize the plot
plt.xlabel("MJD (Modified Julian Date)")
plt.ylabel("Flux [AB]")
plt.title("Light Curves for Different Filters")
plt.legend(title="Filters", ncols=2)
plt.gca().invert_yaxis()  # Invert y-axis for magnitudes (lower mags are brighter)
plt.show()

## Finding the peak brightness and time for each filter

In [ ]:
# Ensuring the data is sorted by time
lc_data.sort('time')

# Finding the peak brightness and time for each filter
peak_info = {}
for filt in filter_names:
    mask = lc_data['band'] == filt
    filtered_data = lc_data[mask]
    peak_flux = np.max(filtered_data['flux'])
    peak_time = filtered_data['time'][np.argmax(filtered_data['flux'])]
    peak_info[filt] = {'flux': peak_flux, 'time': peak_time}
    print(f"Filter: {filt}, Peak Flux: {peak_flux:.2f}, Peak Time: {peak_time:.2f}")

## Computing the rise time and decline rate for each filters

In [ ]:
# Computing rise time and decline rate for all filters
for filt in filter_names:
    mask = lc_data['band'] == filt
    filtered_data = lc_data[mask]
    peak_flux = peak_info[filt]['flux']
    peak_time = peak_info[filt]['time']

    # Computing rise time (time to reach 50% of peak flux)
    pre_peak_mask = filtered_data['time'] < peak_time
    rise_data = filtered_data[pre_peak_mask]
    if len(rise_data) > 0:
        rise_time = peak_time - rise_data['time'][np.argmin(np.abs(rise_data['flux'] - 0.5 * peak_flux))]
        rise_info = f"Rise Time: {rise_time:.2f} days"
    else:
        rise_info = "Rise Time: Insufficient data"

    # Computing decline rate (flux decline 15 days after peak)
    post_peak_mask = (filtered_data['time'] > peak_time) & (filtered_data['time'] <= peak_time + 15)
    post_peak_data = filtered_data[post_peak_mask]
    if len(post_peak_data) > 0:
        flux_15_days = post_peak_data['flux'][-1]
        decline_rate = (peak_flux - flux_15_days) / 15
        decline_info = f"Decline Rate: {decline_rate:.2f} flux units/day"
    else:
        decline_info = "Decline Rate: Insufficient data"

    # Printing both rise time and decline rate in a single line
    print(f"Filter: {filt}, {rise_info}, {decline_info}")

## Fitting the light curves using sncosmo

We now fit our data with a sncosmo.Model used for a Type Ia supernova (using the SALT3 model) to get the light curve parameters. Light curve fitting parameters provide a bridge between observational data and physical or cosmological insights. They are key to understanding transient events, measuring distances, and exploring the nature of the universe.

Fitting parameters include:

 1. \( t_0 \): Time of Maximum Brightness**
- **Definition**: The time (in MJD) when the transient reaches peak brightness.
- **Significance**:
  - Determines the event's timeline.
  - Allows comparison of the light curve evolution with other observations (e.g., spectra).

---

 2. \( x_0 \): Amplitude (Luminosity Parameter)**
- **Definition**: Scaling factor for the brightness of the light curve.
- **Significance**:
  - Related to the absolute magnitude and luminosity of the transient.
  - Used for distance estimation (e.g., standard candles like Type Ia supernovae).

---

 3. \( x_1 \): Stretch (Light Curve Shape)**
- **Definition**: Describes how fast or slow the transient brightens and fades compared to a standard template.
- **Significance**:
  - Indicates explosion energy and nickel-56 production (e.g., brighter events decay slower).
  - Provides insights into the explosion mechanism and progenitor system.

---

 4. \( c \): Color Parameter**
- **Definition**: A measure of color deviation from intrinsic properties, including dust extinction effects.
- **Significance**:
  - Estimates host galaxy dust extinction.
  - Crucial for standardizing brightness for distance measurements.


In [ ]:
# Initialize the sncosmo model (example: salt3_nir for Type Ia Supernovae in the NIR)
model = sncosmo.Model(source='salt2')

# Set initial parameters for the model if required (example: redshift)
model.set(z=1.0)  # Adjust the redshift or other parameters as needed

# Perform light curve fitting
result, fitted_model = sncosmo.fit_lc(
    lc_data,
    model,
    ['z', 't0', 'x0', 'x1', 'c'],  # Parameters to fit
    bounds={
        't0': (lc_data['time'].min(), lc_data['time'].max()),  # Bounds for time of peak
        'x1': (-3e-6, 3),  # Bounds for stretch parameter
        'c': (-3e-6, 3),  # Bounds for color parameter
        'z':(0.01, 1.2) # Bounds for redshift
    },
)

# Plot the original light curve data with the fitted model
sncosmo.plot_lc(lc_data, model=fitted_model, errors=result.errors)
plt.show()

**************
# 3. Color curves <a class="anchor" id="3-CCs"></a>

## Creating color curves

In [ ]:
# Create a dictionary to group fluxes by filter and time
flux_dict = {filt: lc_data['flux'][lc_data['band'] == filt] for filt in filter_names}
time_dict = {filt: lc_data['time'][lc_data['band'] == filt] for filt in filter_names}

# Align times across filters to create color indices
aligned_times = np.unique(np.concatenate([time_dict[filt] for filt in filter_names]))

# Interpolate flux values for each filter at aligned times
interp_flux = {}
for filt in filter_names:
    interp_flux[filt] = np.interp(aligned_times, time_dict[filt], flux_dict[filt])

# # Compute color indices
color_indices = {
    'r-i': interp_flux['f087'] - interp_flux['f106'],  # Example: r-i; change the filter names accordingly
    'g-r': interp_flux['f062'] - interp_flux['f087'],  # Example: g-r; change the filter names accordingly
    'i-z': interp_flux['f106'] - interp_flux['f129']   # Example: i-z; change the filter names accordingly
}

# Uncomment if you are using the sample data from sncosmo
# color_indices = {
#     'r-i': interp_flux['sdssr'] - interp_flux['sdssi'],  # Example: r-i
#     'r-g': interp_flux['sdssr'] - interp_flux['sdssg'],  # Example: r-g
#     'i-z': interp_flux['sdssi'] - interp_flux['sdssz']   # Example: i-z
# }

## Plotting several color curves

In [ ]:
# Plot color curves
plt.figure(figsize=(10, 6))
for color_index, values in color_indices.items():
    plt.scatter(aligned_times, values, label=color_index)

# Customize the plot
plt.xlabel("MJD (Modified Julian Date)")
plt.ylabel("Color Index (Mag)")
plt.title("Color Curves Over Time")
plt.legend(title="Color Index")
plt.grid()
plt.show()


**************
# 4. Additional resources <a class="anchor" id="4-ADs"></a>

- [Roman User Documentation -- RDox](https://roman-docs.stsci.edu/)
- [MAST](https://archive.stsci.edu)
- [ASDF python API](https://asdf.readthedocs.io/en/latest/)
- [ASDF standard](https://asdf-standard.readthedocs.io/)
- [RDox WFI Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-L2ScienceDataSpecifications)
- ["WebbPSF Overview"](https://roman-docs.stsci.edu/simulation-tools-handbook-home/webbpsf-for-roman/overview-of-webbpsf)
- [PSF photometry](https://photutils.readthedocs.io/en/stable/psf.html)
- [Roman instrument model](https://webbpsf.readthedocs.io/en/stable/roman.html)
- [Roman Help Desk](https://roman-docs.stsci.edu/roman-help-desk-at-stsci)

**************
# 5. About this notebook <a class="anchor" id="5-About"></a>

If you have any questions or need further help with the material presented in this notebook, feel free to contact [mshahbandeh@stsci.edu](mshahbandeh@stsci.edu).

**Author:** [Melissa Shahbandeh](https://github.com/shahbandeh/Roman)  

**Updated On:** 2024-10-05

**************
# 6. Citations <a class="anchor" id="6-Cite"></a>

[Open Universe Roman simulations](https://irsa.ipac.caltech.edu/data/theory/openuniverse2024/overview.html)
[SNCosmo](https://zenodo.org/records/14025775)

**************
**************

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 